In [78]:
import pandas as pd

import plotly.express as px

from main import GameResult

In [54]:
df = pd.read_csv("results.csv")

In [74]:
def _sanity_check(x, CAT, animal_cols):
    cat_finished = x[CAT] == GameResult.FINISHED.name
    any_animal_finished = any(x[col] == GameResult.FINISHED.name for col in animal_cols)
    all_animals_chased = all(x[col] == GameResult.CHASED.name for col in animal_cols)
    assert cat_finished != any_animal_finished
    if any_animal_finished or all_animals_chased:
        assert any_animal_finished != all_animals_chased


def get_winner(x):
    CAT = "C"
    animal_cols = ["B", "S", "M"]
    _sanity_check(x, CAT, animal_cols)
    result = None
    for col in [CAT] + animal_cols:
        if x[col] == GameResult.FINISHED.name:
            result = col
            break
    return result

In [75]:
ds = []
for g_id in df.game_id.unique():
    df_game = df[df["game_id"] == g_id]
    last_epoch = df_game.epoch.max()
    df_last_epoch = df_game[df_game["epoch"] == last_epoch]
    d = {"end_epoch": last_epoch, "game_id": g_id}
    for idx, row in df_last_epoch.iterrows():
        d[row["name"]] = GameResult(row["game_result"]).name
    ds.append(d)

df_game_result = pd.DataFrame(ds)
df_game_result["winner"] = df_game_result.apply(get_winner, axis=1)

In [80]:
df_game_result[df_game_result["C"] != GameResult.FINISHED.name]
px.histogram(df_game_result, "winner")


In [ ]:
px.histogram(df_game_result, x="end_epoch", nbins=len(df_game_result.end_epoch.unique()))